In [31]:
# Install the necessary libraries
! pip install pandas dune-client statsmodels causalimpact matplotlib seaborn

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
! pip install dotenv


Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 0/2 [python-dotenv]
   ---------------------------------------- 2/2 [dotenv]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# =========================================================================
# PHASE 1: DATA COLLECTION AND PREPROCESSING (FINAL MULTI-SOURCE VERSION)
# =========================================================================

import pandas as pd
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import os
from dotenv import load_dotenv
import requests
import json
from datetime import datetime, timedelta

# Load environment variables from .env file (for API key)
load_dotenv()

# --- Configuration ---
# NOTE: All Query IDs are now defined and fetched via Dune.
GAS_PRICE_QUERY_ID = 6074978      # Daily Gas Prices (Gwei)
NFT_ACTIVITY_QUERY_ID = 6075187   # Daily NFT Volume & Count
NEW_ADDRESSES_QUERY_ID = 6097091  # Daily New Ethereum Addresses (Confounder)
ETH_PRICE_QUERY_ID = 6075393      # Daily ETH Price in USD (Confounder) - FETCHED VIA DUNE

OUTPUT_DIR = '../data/' # Saves final merged CSV here
RAW_DIR = os.path.join(OUTPUT_DIR, 'raw') # Saves raw CSVs here

# Create necessary directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(RAW_DIR, exist_ok=True)

In [9]:
# --- 1. Dune Data Fetching Function (For On-Chain Data: Gas, NFT, New Addresses, ETH Price) ---
def fetch_dune_data(query_id: int, query_name: str, file_name: str) -> pd.DataFrame:
    """Fetches the latest result from a Dune query ID and saves it to the RAW directory."""
    try:
        dune = DuneClient(os.environ["DUNE_API_KEY"]) 
        query = QueryBase(query_id=query_id, name=query_name)
        
        print(f"Executing Dune Query: {query_name} (ID: {query_id})...")
        df = dune.run_query_dataframe(query)
        
        # Save raw data to data/raw/
        raw_path = os.path.join(RAW_DIR, file_name)
        df.to_csv(raw_path, index=False)
        print(f"   SUCCESS: Fetched {len(df)} rows. Saved to {raw_path}")
        return df
    except KeyError:
        print("   ERROR: DUNE_API_KEY environment variable not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"   FAILED: Error fetching data from Dune for {query_name}: {e}")
        return pd.DataFrame()





In [13]:
import pandas as pd
import requests
import os

def fetch_fg_index_data() -> pd.DataFrame:
    """Fetches historical Fear & Greed Index data using the stable JSON API (UTC formatted with millisecond precision)."""
    fg_url = "https://api.alternative.me/fng/?limit=0" 
    print("\n[STEP 1] Fetching Fear & Greed Index (Score and Classification)...")

    try:
        response = requests.get(fg_url)
        response.raise_for_status()
        
        data = response.json()['data']
        df = pd.DataFrame(data)
        
        # ✅ Convert Unix timestamp → UTC datetime (normalize to midnight)
        df['date'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)
        df['date'] = df['date'].dt.floor('D')
        
        # ✅ Format to: YYYY-MM-DD HH:MM:SS.mmm UTC (3 decimal places)
        df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S.%f').str[:-3] + ' UTC'
        
        # ✅ Keep only relevant columns
        df = df[['date', 'value', 'value_classification']].rename(
            columns={'value': 'fear_greed_score', 'value_classification': 'fear_greed_rating'}
        )
        
        # ✅ Ensure numeric score
        df['fear_greed_score'] = pd.to_numeric(df['fear_greed_score'], errors='coerce')
        df = df.dropna(subset=['fear_greed_score'])
        
        # ✅ Save to CSV (no index)
        raw_path = os.path.join(RAW_DIR, "fg_index_raw.csv")
        df.to_csv(raw_path, index=False)

        print(f"   SUCCESS: Fetched and saved {len(df)} days of Fear & Greed data (UTC, ms precision).")
        return df

    except Exception as e:
        print(f"   FAILED: Error fetching F&G Index via JSON API: {e}")
        return pd.DataFrame()


In [17]:
! pip install yfinance

! pip install backtrader[plotting] 

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ----------- ---------------------------- 262.1/949.2 kB ? eta -:--:--
     ------------------------------- ------ 786.4/949.2 kB 2.2 MB/s eta 0:00:01
     ---------------------------------------- 949.2/949.2 kB 2.1 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# --- Main Execution ---

# 1. Fetch/Load ALL Data Sources
df_gas = fetch_dune_data(GAS_PRICE_QUERY_ID, "Daily Gas Prices", "gas_prices_raw.csv")
df_nft = fetch_dune_data(NFT_ACTIVITY_QUERY_ID, "Daily NFT Activity", "nft_sales_raw.csv")
df_new_addresses = fetch_dune_data(NEW_ADDRESSES_QUERY_ID, "Daily New Addresses", "new_addresses_raw.csv")
df_eth_price = fetch_dune_data(ETH_PRICE_QUERY_ID, "Daily ETH Price", "eth_price_raw.csv") # <-- CORRECTED FETCH

 # External API fetch



2025-11-01 10:21:30,079 INFO dune_client.api.base executing 6074978 on medium cluster


Executing Dune Query: Daily Gas Prices (ID: 6074978)...


2025-11-01 10:21:32,047 INFO dune_client.api.base waiting for query execution 01K8YW394ZQ4BVC0T7315HRXRG to complete: ExecutionState.PENDING (queue position: None)
2025-11-01 10:21:33,236 INFO dune_client.api.base waiting for query execution 01K8YW394ZQ4BVC0T7315HRXRG to complete: ExecutionState.PENDING (queue position: None)
2025-11-01 10:21:34,412 INFO dune_client.api.base waiting for query execution 01K8YW394ZQ4BVC0T7315HRXRG to complete: ExecutionState.PENDING (queue position: None)
2025-11-01 10:21:35,594 INFO dune_client.api.base waiting for query execution 01K8YW394ZQ4BVC0T7315HRXRG to complete: ExecutionState.EXECUTING
2025-11-01 10:21:36,771 INFO dune_client.api.base waiting for query execution 01K8YW394ZQ4BVC0T7315HRXRG to complete: ExecutionState.EXECUTING
2025-11-01 10:21:37,937 INFO dune_client.api.base waiting for query execution 01K8YW394ZQ4BVC0T7315HRXRG to complete: ExecutionState.EXECUTING
2025-11-01 10:21:39,105 INFO dune_client.api.base waiting for query execution 0

   SUCCESS: Fetched 181 rows. Saved to ../data/raw\gas_prices_raw.csv
Executing Dune Query: Daily NFT Activity (ID: 6075187)...


2025-11-01 10:21:46,998 INFO dune_client.api.base waiting for query execution 01K8YW3QRMV3MY978YMT27J658 to complete: ExecutionState.PENDING (queue position: None)
2025-11-01 10:21:48,167 INFO dune_client.api.base waiting for query execution 01K8YW3QRMV3MY978YMT27J658 to complete: ExecutionState.PENDING (queue position: None)
2025-11-01 10:21:49,339 INFO dune_client.api.base waiting for query execution 01K8YW3QRMV3MY978YMT27J658 to complete: ExecutionState.EXECUTING
2025-11-01 10:21:50,782 INFO dune_client.api.base executing 6097091 on medium cluster


...

   SUCCESS: Fetched 181 rows. Saved to ../data/raw\new_addresses_raw.csv
Executing Dune Query: Daily ETH Price (ID: 6075393)...


2025-11-01 10:30:16,006 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: ExecutionState.PENDING (queue position: None)
2025-11-01 10:30:17,181 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: ExecutionState.EXECUTING
2025-11-01 10:30:18,367 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: ExecutionState.EXECUTING
2025-11-01 10:30:19,540 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: ExecutionState.EXECUTING
2025-11-01 10:30:20,712 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: ExecutionState.EXECUTING
2025-11-01 10:30:21,892 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: ExecutionState.EXECUTING
2025-11-01 10:30:23,065 INFO dune_client.api.base waiting for query execution 01K8YWK8T2MJCRMEXM4XJ7S81X to complete: Exe

   SUCCESS: Fetched 180 rows. Saved to ../data/raw\eth_price_raw.csv

[STEP 1] Fetching Fear & Greed Index (Score and Classification)...


C:\Users\Devi\AppData\Local\Temp\ipykernel_25040\4072048170.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.normalize()


   SUCCESS: Fetched and saved 2827 days of Fear & Greed data.


In [14]:
df_fg_index = fetch_fg_index_data()


[STEP 1] Fetching Fear & Greed Index (Score and Classification)...
   SUCCESS: Fetched and saved 2827 days of Fear & Greed data (UTC, ms precision).


C:\Users\Devi\AppData\Local\Temp\ipykernel_25040\364121087.py:18: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)


In [17]:
import pandas as pd
import os

RAW_DIR = "../data/raw"
OUTPUT_DIR = "../data"

files = {
    "gas": f"{RAW_DIR}/gas_prices_raw.csv",
    "nft": f"{RAW_DIR}/nft_sales_raw.csv",
    "eth_price": f"{RAW_DIR}/eth_price_raw.csv",
    "new_addr": f"{RAW_DIR}/new_addresses_raw.csv",
    "fg_index": f"{RAW_DIR}/fg_index_raw.csv"
}

dfs = {}
for name, path in files.items():
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'], utc=True, errors='coerce')
    dfs[name] = df

# --- Merge on 'date' using outer join ---
merged_ts = dfs['gas']
for key in ['nft', 'eth_price', 'new_addr', 'fg_index']:
    merged_ts = pd.merge(merged_ts, dfs[key], on='date', how='outer')

# Sort and clean
merged_ts = merged_ts.sort_values('date').reset_index(drop=True)

# Save
output_path = os.path.join(OUTPUT_DIR, "merged_timeseries.csv")
merged_ts.to_csv(output_path, index=False)

print(f"\n✅ Merged {len(merged_ts)} rows saved to {output_path}")
print("Date range:", merged_ts['date'].min(), "→", merged_ts['date'].max())
print("Missing values per column:\n", merged_ts.isna().sum())



✅ Merged 2827 rows saved to ../data\merged_timeseries.csv
Date range: 2018-02-01 00:00:00+00:00 → 2025-11-01 00:00:00+00:00
Missing values per column:
 date                        0
avg_gas_price_gwei       2646
nft_sales_volume_eth     2646
nft_transaction_count    2646
eth_price_usd            2647
new_addresses            2646
fear_greed_score            0
fear_greed_rating           0
dtype: int64


In [18]:
import pandas as pd

# Load your merged file
merged_ts = pd.read_csv("../data/merged_timeseries.csv")

# Convert date column to datetime (just to be safe)
merged_ts['date'] = pd.to_datetime(merged_ts['date'], utc=True, errors='coerce')

# ✅ Keep only rows where all key datasets have data
merged_clean = merged_ts.dropna(subset=[
    'avg_gas_price_gwei',
    'nft_sales_volume_eth',
    'eth_price_usd',
    'new_addresses'
])

# Sort by date
merged_clean = merged_clean.sort_values('date').reset_index(drop=True)

# Save the cleaned overlapping dataset
output_path = "../data/merged_timeseries_clean.csv"
merged_clean.to_csv(output_path, index=False)

print(f"✅ Filtered dataset saved to: {output_path}")
print(f"Rows after filtering: {len(merged_clean)}")
print("Date range:", merged_clean['date'].min(), "→", merged_clean['date'].max())


✅ Filtered dataset saved to: ../data/merged_timeseries_clean.csv
Rows after filtering: 180
Date range: 2025-05-06 00:00:00+00:00 → 2025-11-01 00:00:00+00:00


In [20]:


def impute_outliers_and_finalize_data(input_path: str, output_path: str) -> pd.DataFrame:
    """
    Loads the merged and date-aligned data, applies IQR-based median imputation
    to handle volatility spikes, and prepares the final time-series dataset.
    """
    print("\n[STEP 3] Starting Outlier Imputation and Finalization...")
    
    # 1. Load the pre-filtered, aligned data
    df = pd.read_csv(input_path, parse_dates=['date'])
    df = df.set_index('date').sort_index()

    # Define the columns that are highly volatile and need capping/imputation
    volatile_cols = [
        'nft_sales_volume_eth', 
        'avg_gas_price_gwei', 
        'new_addresses', 
        'fear_greed_score'
    ]

    def cap_and_impute_outliers(df, column):
        """
        Identifies extreme outliers (3.0 * IQR) and replaces them 
        with the column's local median.
        """
        if column not in df.columns:
            print(f"   WARNING: Column '{column}' not found. Skipping imputation for this variable.")
            return df
            
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        # Define the bounds for extreme outliers (3 * IQR)
        upper_bound = Q3 + 3.0 * IQR
        lower_bound = Q1 - 3.0 * IQR
        
        outliers = (df[column] > upper_bound) | (df[column] < lower_bound)
        num_outliers = outliers.sum()
        
        if num_outliers > 0:
            # Calculate the median of the non-outlier data and replace
            median_value = df.loc[~outliers, column].median()
            df.loc[outliers, column] = median_value
            print(f"   Handled {num_outliers} extreme outliers in '{column}'.")
        
        return df

    # 2. Apply Outlier Handling to all volatile columns
    for col in volatile_cols:
        df = cap_and_impute_outliers(df, col)

    # 3. Finalize Dataset
    
    # Ensure Index is datetime for time functions
    df.index = pd.to_datetime(df.index)
    
    # Add Day of Week (Temporal Confounder)
    df['day_of_week'] = df.index.day_name()
    
    # Final cleanup (just in case) and save
    df_final = df.dropna()
    df_final.to_csv(output_path, index=True)

    print(f"\n✅ Final imputed dataset saved to: {output_path}")
    print(f"Rows in final dataset: {len(df_final)}")
    
    return df_final


# --- Execution Example ---
input_file = "../data/merged_timeseries_clean.csv" # The output of your filtering step
output_file = "../data/merged_timeseries_final_imputed.csv"

# Run the function on your already filtered file
final_imputed_df = impute_outliers_and_finalize_data(input_file, output_file)

# The 'merged_timeseries_final_imputed.csv' is now ready for Notebook 3 (Modeling)


[STEP 3] Starting Outlier Imputation and Finalization...
   Handled 28 extreme outliers in 'nft_sales_volume_eth'.
   Handled 3 extreme outliers in 'avg_gas_price_gwei'.
   Handled 2 extreme outliers in 'new_addresses'.

✅ Final imputed dataset saved to: ../data/merged_timeseries_final_imputed.csv
Rows in final dataset: 180
